In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver as wd
import pandas as pd
import numpy as np
import time

In [5]:
driver = wd.Chrome()
def scroll(driver, timeout):
    scroll_pause_time = timeout
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(scroll_pause_time)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height

In [6]:
titles, years, certificates, runtimes, genres, ratings, metascores = [], [], [], [], [], [], []
data = [titles, years, certificates, runtimes, genres, ratings, metascores]

In [7]:
pages = np.arange(1,1000, 100)
for i in range (len(pages)):
    link = "https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(pages[i])+"&ref_=adv_nxt"
    driver.get(link)
    scroll(driver, 1)
    content = driver.page_source
    soup = BeautifulSoup(content)
    for a in soup.findAll('div', attrs={'class':'lister-item-content'}):
        x = []
        x.append(a.find('a', href=True))
        x.append(a.find('span', attrs={'class':'lister-item-year text-muted unbold'}))
        x.append(a.find('span', attrs={'class':'certificate'}))
        x.append(a.find('span', attrs={'class':'runtime'}))
        x.append(a.find('span', attrs={'class':'genre'}))
        x.append(a.find('div', attrs={'class':'inline-block ratings-imdb-rating'}))
        x.append(a.find('div', attrs={'class':'inline-block ratings-metascore'}))
        for i in range(len(x)):
            if isinstance(x[i], type(None)):
                data[i].append(None)
            else:
                x_value = x[i].text.replace("\n","")
                data[i].append(x_value)

In [8]:
for i in range(len(data[6])):
    val_genre = data[4][i].split()
    data[4][i] = ""
    for j in val_genre:
        data[4][i]  += j + " "
    if(type(data[6][i]) != type(None)):
        val = data[6][i].split()
        data[6][i] = val[0]

In [10]:
attributes = ['title', 'year', 'certificate', 'runtime','genre','rating','metascore']
df = pd.DataFrame()
for i in range(len(attributes)):
    df.insert(len(df.columns), attributes[i], data[i]) 
df

,title,year,certificate,runtime,genre,rating,metascore
0,The Shawshank Redemption,(1994),R,142 min,Drama,9.3,80
1,The Godfather,(1972),R,175 min,"Crime, Drama",9.2,100
2,The Dark Knight,(2008),PG-13,152 min,"Action, Crime, Drama",9.0,84
3,The Godfather: Part II,(1974),R,202 min,"Crime, Drama",9.0,90
4,12 Angry Men,(1957),Approved,96 min,"Crime, Drama",9.0,96
...,...,...,...,...,...,...,...
995,Giant,(1956),G,201 min,"Drama, Western",7.6,84
996,From Here to Eternity,(1953),Passed,118 min,"Drama, Romance, War",7.6,85
997,Lifeboat,(1944),Not Rated,97 min,"Drama, War",7.6,78
998,Snow White and the Seven Dwarfs,(1937),Approved,83 min,"Animation, Family, Fantasy",7.6,95


In [11]:
df.to_csv('movie_data.csv', index=False, encoding='utf-8')
driver.close()